<a href="https://colab.research.google.com/github/pratikchaudhari64/notebooks/blob/main/pytorch_and_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)


100%|██████████| 26.4M/26.4M [00:01<00:00, 17.5MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 308kB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 5.59MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 8.70MB/s]


In [8]:
input_image = torch.rand(3,28,28)
# print(input_image.size())
input_image

tensor([[[0.3719, 0.9587, 0.4808,  ..., 0.6046, 0.5051, 0.4253],
         [0.1208, 0.0835, 0.9386,  ..., 0.0025, 0.1693, 0.3567],
         [0.6199, 0.4819, 0.9606,  ..., 0.5264, 0.8035, 0.7195],
         ...,
         [0.0412, 0.6606, 0.0607,  ..., 0.4508, 0.1371, 0.6911],
         [0.9271, 0.9387, 0.9283,  ..., 0.2669, 0.4915, 0.8297],
         [0.6763, 0.8456, 0.2497,  ..., 0.5438, 0.1006, 0.2780]],

        [[0.6630, 0.7683, 0.7723,  ..., 0.1316, 0.3116, 0.5111],
         [0.5591, 0.4893, 0.1073,  ..., 0.7623, 0.6717, 0.3138],
         [0.7144, 0.6838, 0.3237,  ..., 0.1913, 0.3808, 0.8168],
         ...,
         [0.5154, 0.6553, 0.7767,  ..., 0.8166, 0.7574, 0.1195],
         [0.2924, 0.9452, 0.1312,  ..., 0.7280, 0.7580, 0.9035],
         [0.0140, 0.2385, 0.2147,  ..., 0.1624, 0.7470, 0.1926]],

        [[0.7165, 0.0475, 0.0730,  ..., 0.2792, 0.9303, 0.7659],
         [0.5106, 0.2563, 0.4318,  ..., 0.7198, 0.6428, 0.2937],
         [0.1503, 0.5338, 0.1091,  ..., 0.5814, 0.5106, 0.

In [ ]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())